# Assignment 1
In this assignment you will be creating tools for learning and testing language models.
The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.


In [2]:
import os
import io
import sys
import pandas as pd
import numpy as np
import math
from sklearn.metrics import f1_score

start_token = '<start>'
end_token = '<end>'
languages = ["en","es","in","it","pt","fr","nl","tl"]
def updateTweet(tweet):
  return "בבב" +tweet+"בבב"

*As a preparation for this task, download the data files from the course git repository.

The relevant files are under **lm-languages-data-new**:


*   en.csv (or the equivalent JSON file)
*   es.csv (or the equivalent JSON file)
*   fr.csv (or the equivalent JSON file)
*   in.csv (or the equivalent JSON file)
*   it.csv (or the equivalent JSON file)
*   nl.csv (or the equivalent JSON file)
*   pt.csv (or the equivalent JSON file)
*   tl.csv (or the equivalent JSON file)
*   test.csv (or the equivalent JSON file)





In [3]:
!git clone https://github.com/kfirbar/nlp-course.git

fatal: destination path 'nlp-course' already exists and is not an empty directory.




---



**Important note: please use only the files under lm-languages-data-new and NOT under lm-languages-data**


---



In [4]:

!ls nlp-course/lm-languages-data-new


'ls' is not recognized as an internal or external command,
operable program or batch file.


**Part 1**

Write a function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. **Our token definition is a single UTF-8 encoded character**. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data.

In [14]:
data_files = {'en_df': 'en.csv',
              'es_df': 'es.csv',
              'fr_df': 'fr.csv',
              'in_df': 'in.csv',
              'it_df': 'it.csv',
              'nl_df': 'nl.csv',
              'pt_df': 'pt.csv',
              'tl_df': 'tl.csv'}
data_files = {'en_df': r'C:\MSC\NLP2\nlp-course\lm-languages-data\en.csv',
              'es_df': r'C:\MSC\NLP2\nlp-course\lm-languages-data\es.csv',
              'fr_df': r'C:\MSC\NLP2\nlp-course\lm-languages-data\fr.csv',
              'in_df': r'C:\MSC\NLP2\nlp-course\lm-languages-data\in.csv',
              'it_df': r'C:\MSC\NLP2\nlp-course\lm-languages-data\it.csv',
              'nl_df': r'C:\MSC\NLP2\nlp-course\lm-languages-data\nl.csv',
              'pt_df': r'C:\MSC\NLP2\nlp-course\lm-languages-data\pt.csv',
              'tl_df': r'C:\MSC\NLP2\nlp-course\lm-languages-data\tl.csv'}
directory = 'nlp-course/lm-languages-data-new/'
start_token = '↠'
end_token = '↞'

In [15]:
import pandas as pd
def preprocess():
  tokens = {}
  for language in languages:
    tweets =  pd.read_csv(f'nlp-course/lm-languages-data-new/{language}.csv').get('tweet_text')
    for tweet in tweets:
      for char in tweet:
        tokens[char]=0
  return list(tokens.keys())




**Part 2**

Write a function lm that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant n-1 sequences, and the values are dictionaries with the n_th tokens and their corresponding probabilities to occur. For example, for a trigram model (tokens are characters), it should look something like:

{
  "ab":{"c":0.5, "b":0.25, "d":0.25},
  "ca":{"a":0.2, "b":0.7, "d":0.1}
}

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

In [16]:
def lm(n, vocabulary, data_file_path, add_one):
  # n - the n-gram to use (e.g., 1 - unigram, 2 - bigram, etc.)
  # vocabulary - the vocabulary list (which you should use for calculating add_one smoothing)
  # data_file_path - the data_file from which we record probabilities for our model
  # add_one - True/False (use add_one smoothing or not)
  if add_one:
      add_one = 1
  else:
      add_one = 0
  tweets =  pd.read_csv(data_file_path).get('tweet_text')
  model = {}
  for tweet in tweets:
    if n > 1:
      tweet = updateTweet(tweet)
    startIndexs =range(len(tweet) - n + 1)
    endIndexs = range(n-1, len(tweet))
    nGram = zip(startIndexs, endIndexs)
  
    for i,j in nGram:
      wNMinusOne = tweet[i: i + n -1]
      if i==j:
          wNMinusOne = ""
      wN = tweet[j]
      new_count = model.setdefault(wNMinusOne, {}).setdefault(wN, 0) + 1
      model[wNMinusOne][wN] = new_count

  for prefix in model.values():
      countPrefixW = float(sum(prefix.values()))
      if add_one:
        countPrefixW += len(vocabulary)
      for key in prefix.keys():
        prefix[key] = (prefix[key] + add_one) / countPrefixW  
  return model

**Part 3**

Write a function *eval* that returns the perplexity of a model (dictionary) running over a given data file.

In [17]:
def eval(n, model, data_file):
  # n - the n-gram that you used to build your model (must be the same number)
  # model - the dictionary (model) to use for calculating perplexity
  # data_file - the tweets file that you wish to claculate a perplexity score for
  

  tweets =  pd.read_csv(data_file).get('tweet_text')
  entropyList=[]
  for tweet in tweets:
    entropyList.append(evalTweet(tweet,n,model))

  return np.array(entropyList).mean()

def evalTweet(tweet,n, model):
    missing_value = 1e-8
    tweet = updateTweet(tweet)
    startIndexs =range(len(tweet) - n + 1)
    endIndexs = range(n-1, len(tweet))
    nGram = zip(startIndexs, endIndexs)
    N=0
    entropy =0
    for i,j in nGram:
      wNMinusOne = '' 
      N+=1
      if i!=j:
          wNMinusOne = tweet[i: i + n -1]
      wN = tweet[j]
      pwN = missing_value
      if wNMinusOne in model.keys():
             if wN in model[wNMinusOne].keys():
               entropy+= -math.log2(model[wNMinusOne][wN])
             else:
              entropy+=-math.log2(missing_value)
      else:
          entropy+=-math.log2(missing_value)
    entropy = entropy/N
    return math.pow(2,entropy)

In [18]:
vocabulary = preprocess()
n = 2
test_dict = lm(n, vocabulary, data_files['en_df'], False)
eval(n,test_dict, data_files['en_df'])

16.996093661707146

**Part 4**

Write a function *match* that creates a model for every relevant language, using a specific value of *n* and *add_one*. Then, calculate the perplexity of all possible pairs (e.g., en model applied on the data files en ,es, fr, in, it, nl, pt, tl; es model applied on the data files en, es...). This function should return a pandas DataFrame with columns [en ,es, fr, in, it, nl, pt, tl] and every row should be labeled with one of the languages. Then, the values are the relevant perplexity values.

In [147]:
def match(n, add_one):
  # n - the n-gram to use for creating n-gram models
  # add_one - use add_one smoothing or not
  model= {}
  df = {}

  for language_model in languages:
    data_file_path = f'nlp-course/lm-languages-data-new/{language_model}.csv'
    model[language_model] = lm(n, preprocess(),data_file_path,add_one)
    df[language_model] = {}
    for language in languages:
       data_file_path = f'nlp-course/lm-languages-data-new/{language}.csv'
       df[language_model][language]=eval(n,model[language_model],data_file_path)
  
  return pd.DataFrame(df)

   

  

**Part 5**

Run match with *n* values 1-4, once with add_one and once without, and print the 8 tables to this notebook, one after another.

In [142]:
def run_match():
  for i in range(1,5):
    print(f"add_one = True and n = {i}")
    print(match(i,True))
    print(f"add_one = False and n = {i}")
    print(match(i,False))
    

run_match()

add_one = True and n = 1


NameError: ignored

**Part 6**

Each line in the file test.csv contains a sentence and the language it belongs to. Write a function that uses your language models to classify the correct language of each sentence.

Important note regarding the grading of this section: this is an open question, where a different solution will yield different accuracy scores. any solution that is not trivial (e.g. returning 'en' in all cases) will be excepted. We do reserve the right to give bonus points to exceptionally good/creative solutions.

In [151]:
def classify():
   models = {}
   res = []
   n=3
   add_one =True
   data_file_path = f'nlp-course/lm-languages-data-new/test.csv'
   for language_model in languages:
        models[language_model] = lm(n, preprocess(),f'nlp-course/lm-languages-data-new/{language_model}.csv',add_one)
   tweets =  pd.read_csv(data_file_path).get('tweet_text')
   for tweet in tweets:
     correctLang = languages[0]
     minperplexity = float('inf')
     for lang, model in models.items():
        perplexity = evalTweet(tweet,n,model)
        if perplexity< minperplexity:
          correctLang = lang
          minperplexity = perplexity
     res.append(correctLang)
   return res




   

  # TODO
clasification_result = classify()
print(clasification_result)

['en', 'it', 'tl', 'nl', 'tl', 'in', 'pt', 'nl', 'fr', 'pt', 'en', 'nl', 'pt', 'en', 'es', 'fr', 'en', 'tl', 'fr', 'en', 'it', 'fr', 'en', 'pt', 'tl', 'pt', 'en', 'fr', 'fr', 'in', 'nl', 'tl', 'nl', 'tl', 'it', 'nl', 'in', 'es', 'es', 'nl', 'fr', 'en', 'es', 'fr', 'it', 'es', 'nl', 'es', 'tl', 'tl', 'en', 'nl', 'pt', 'tl', 'nl', 'en', 'pt', 'tl', 'pt', 'en', 'pt', 'pt', 'tl', 'en', 'en', 'tl', 'tl', 'nl', 'it', 'tl', 'fr', 'pt', 'in', 'pt', 'in', 'fr', 'pt', 'tl', 'es', 'tl', 'en', 'tl', 'en', 'in', 'nl', 'fr', 'nl', 'in', 'fr', 'tl', 'en', 'pt', 'es', 'nl', 'tl', 'nl', 'tl', 'tl', 'it', 'pt', 'in', 'pt', 'in', 'en', 'es', 'nl', 'pt', 'en', 'pt', 'it', 'es', 'fr', 'pt', 'nl', 'en', 'fr', 'tl', 'it', 'es', 'fr', 'nl', 'en', 'en', 'pt', 'it', 'es', 'nl', 'nl', 'pt', 'es', 'pt', 'it', 'tl', 'pt', 'en', 'es', 'fr', 'es', 'it', 'en', 'fr', 'fr', 'in', 'it', 'es', 'es', 'in', 'tl', 'pt', 'pt', 'pt', 'in', 'in', 'en', 'fr', 'fr', 'it', 'tl', 'nl', 'it', 'es', 'fr', 'fr', 'nl', 'it', 'es', 'nl

**Part 7**

Calculate the F1 score of your output from part 6. (hint: you can use https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html). 


In [153]:
def calc_f1(result):
     data_file_path = f'nlp-course/lm-languages-data-new/test.csv'
     labels = pd.read_csv(data_file_path).get('label')
     print(list(labels))
     return f1_score(list(labels), clasification_result,average="micro")

  # TODO

calc_f1(clasification_result)

['en', 'it', 'tl', 'nl', 'tl', 'in', 'pt', 'nl', 'fr', 'pt', 'en', 'tl', 'pt', 'es', 'es', 'fr', 'en', 'nl', 'fr', 'nl', 'es', 'fr', 'en', 'pt', 'tl', 'pt', 'en', 'fr', 'fr', 'in', 'nl', 'tl', 'nl', 'tl', 'it', 'nl', 'in', 'es', 'es', 'nl', 'fr', 'en', 'es', 'fr', 'it', 'es', 'nl', 'es', 'tl', 'tl', 'en', 'nl', 'pt', 'tl', 'nl', 'en', 'pt', 'tl', 'it', 'en', 'pt', 'it', 'tl', 'fr', 'en', 'tl', 'tl', 'nl', 'it', 'tl', 'fr', 'pt', 'in', 'pt', 'in', 'nl', 'pt', 'tl', 'es', 'tl', 'en', 'tl', 'en', 'in', 'nl', 'fr', 'nl', 'in', 'fr', 'tl', 'en', 'pt', 'es', 'nl', 'tl', 'nl', 'tl', 'tl', 'it', 'es', 'in', 'pt', 'in', 'en', 'es', 'nl', 'pt', 'en', 'pt', 'it', 'es', 'in', 'pt', 'nl', 'tl', 'fr', 'tl', 'it', 'es', 'fr', 'nl', 'en', 'en', 'pt', 'pt', 'es', 'nl', 'en', 'pt', 'es', 'pt', 'it', 'tl', 'it', 'es', 'es', 'fr', 'pt', 'it', 'en', 'fr', 'fr', 'in', 'it', 'es', 'es', 'in', 'tl', 'pt', 'pt', 'pt', 'in', 'in', 'en', 'fr', 'fr', 'it', 'tl', 'nl', 'it', 'es', 'fr', 'fr', 'nl', 'it', 'es', 'nl

0.8723590448806101

# **Good luck!**